# Test technique pour le stage data science chez Bouygues Telecom

<div style="text-align: right;"><span style="font-style: italic; margin-right: 1.5em;">Gabriel Watkinson</span></div> 
<hr class="solid">

Ce notebook contient le test technique pour le stage de data science chez Bouygues Telecom basée sur des données factices sur la résiliation des clients ayant un abonnement télécom.

### Description du test

L'objectif est de comprendre et de prédire la résilation des clients. Pour ce faire 4 datasets sont fournis :

* resiliation_option : contient les données sur les options des forfaits fixe et mobile des clients
* resiliation_client : contient des données clients
* resiliation_contrat : contient les caractéristiques des contrats des clients
* resiliation_forfait : contient les caractéristiques du forfait mobile du client

### Organisation du notebook

Le notebook est composé de 3 parties et d'une conclusion :

1. Analyses des données et statistiques descriptives
2. Préparation de données
3. Modélisation du problème

## Analyses des données et statistiques descriptives

Dans un premier temps, nous analyserons les données pour comprendre leur contenu et leur structure en utilisant des statistiques descriptives et des graphiques pour visualiser les distributions des données, ainsi que les problèmes éventuels, comme les données manquantes par exemple, qui devront être pris en compte lors de la modélisation.

### EDA automatique avec `pandas_profiling`

Pour ce faire, nous allons utiliser `pandas_profiling` qui permet d'explorer les données et de soulever certain problèmes de manière automatique. Puis nous allons regarder manuellement certaines corrélations. Cela va créer des profils avec des informations sur chaques variables (moyenne, médiane, valeurs manquantes, etc.), mais aussi sur quelques intéractions entre les variables (corrélations, etc.).

Ces profils sont ensuite exportés en format html dans le dossier nommé `profiles`.

Les données sont chargées à partir du dossier `data` qui se situe dans la racine du projet (comme ce notebook).

In [ ]:
import pandas as pd

# Chargement des données situées dans le dossier `data/`
data_folder = "./data/"
client = pd.read_csv(data_folder + "resiliation_client.csv", sep=";")
contrat = pd.read_csv(data_folder + "resiliation_contrat.csv", sep=";")
forfait = pd.read_csv(data_folder + "resiliation_forfait.csv", sep=";")
option = pd.read_csv(data_folder + "resiliation_option.csv", sep=";")

In [ ]:
import os
import warnings

warnings.filterwarnings("ignore")

from pandas_profiling import ProfileReport

# Création des profils automatiques
# Création du dossier `profils` qui contient les profils en html
os.makedirs("profils", exist_ok=True)

print("Pour la table client :")
profil_client = ProfileReport(client, title="Profil de la table client")
profil_client.to_file("profils/profil_client.html")

print("Pour la table contrat :")
profil_contrat = ProfileReport(contrat, title="Profil de la table contrat")
profil_contrat.to_file("profils/profil_contrat.html")

print("Pour la table option :")
profil_option = ProfileReport(option, title="Profil de la table option")
profil_option.to_file("profils/profil_option.html")

### Profil `resiliation_client`

Nous commençons avec la table qui contient les données clients.

In [ ]:
profil_client.to_notebook_iframe()  # Affiche le profil dans une iframe

La table `resiliation_client` contient des données personnelles sur les clients, à savoir leur genre, leur situation familliale (en couple, parent) et une indication sur leur âge. Il y a également une information sur leur ancienneté chez Bouygues Telecom (en mois je suppose).

Le profil donne un certain nombre d'informations interessantes :

* Il y a un total de 7043 observations et 7043 identifiants de clients différents. L'id_client est donc une clé primaire qui peut être utilisée pour joindre les tables.
* L'ancienneté a une distribution assez particulière :
  * Avec une forte concentration sur les clients avec une très faible ancienneté, avec 1062 des clients qui ont moins de 3 mois d'ancienneté, soit 15% du nombre total. On note que l'ancienneté diminue assez vite les premiers mois avant de se stabiliser.
  * Mais aussi sur les clients très anciens, avec 362 clients qui ont 72 mois d'ancienneté, soit 9% des clients. Il faut noter que 72 mois est le maximum recensé dans le dataset. On peut supposer que 72 correspond aux clients qui ont plus de 72 mois d'ancienneté. Cependant, on voit tout de même que 71 et 70 sont des valeurs très fréquentes. Il y a peut être une autre raison.
  * Finalement, on peut noter des pics tous les 2 à 3 mois. Il pourrait être important de s'y intéresser, pour savoir si c'est notamment dû à des campagnes de marketing ou à des changements de tarifs par exemple.
* Il y a un total de 4305 observations manquantes soit 10.2% des données. Ces données manquantes sont réparties entre le genre des clients (où 10% des données sont manquantes) et leur situation parentale (où 51% des données sont manquantes). On peut voir que l'absence de la situation parentale n'implique pas l'absence du genre, et inversement. Les absences de ces variables ne sont pas corrélées.

De plus, il peut être intéressant de noter la forte corrélation entre le fait d'être en couple et la situation parentale. En effet, le graphique suivant montre que, parmi les clients en couple, la moitié sont parents, alors que parmi les clients qui ne sont pas en couple, la grosse majorité n'est pas parent.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
%matplotlib inline

ax = sns.countplot(data=client.fillna({"parent": "NaN"}), x="couple", hue="parent")
ax.set(
    xlabel="Couple",
    ylabel="Nombre de clients",
    title="Relation entre être en couple et être parent",
)
ax.legend(title="Parent")
plt.show()

De même, on voit clairement que les clients les plus anciens sont plus souvent en couple que les nouveaux clients. On peut donc penser que les clients célibataires sont plus enclins à résilier leur abonnement alors que les clients en couples le font moins souvent.

In [ ]:
ax = sns.kdeplot(data=client, x="anciennete", hue="couple", cut=0, multiple="fill")
ax.set(
    xlabel="Ancienneté",
    ylabel="Pourcentage des clients",
    title="Proportion des clients en couple selon l'ancienneté",
)
plt.show()

### Profil `resiliation_forfait`

Pour la table avec les informations sur les forfaits, il n'est pas nécessaire de faire un profil. On peut simplement regarder les données car il n'y a que quelques lignes.

En effet, cette table contient uniquement la quantité de Go par mois pour les différents forfaits mobile.

In [ ]:
forfait

### Profil `resiliation_contrat`

Regardons ensuite la table `resiliation_contrat` qui contient les informations sur les contrats des clients.

In [ ]:
profil_contrat.to_notebook_iframe()  # Affiche le profil dans une iframe

La table `resiliation_contrat` contient les données sur les contrats souscrits par les clients, à savoir le forfait, la durée du contrat (mensuel, annuel, biannuel), la méthode de paiement, la facture mensuelle moyenne, la facture totale et surtout si le client a résilé son abonnement ou non. 

Le profil donne un certain nombre d'informations interessantes :

* Comme dans la table sur les clients, l'id_client est une clé primaire. On vérifiera dans la suite que ce sont bien les mêmes clients qui sont présents dans les deux tables.
* On peut voir qu'il y a seulement 4 types de forfaits dans cette table, alors que la table sur les forfait en contient 5. Il y en a donc un qui n'est pas utilisé.
* Il y a moins d'observations manquantes que dans la table sur les clients, avec 9.7% des forfaits manquants.
* Il est très important de remarquer que le nombre d'observations avec résiliation est beaucoup plus faible que celui sans résiliation. Les classes de la *target* ne sont donc pas équilibrées. Il faudra donc y faire attention en choisissant une métrique de classification pertinente ou en appliquant des méthodes de *sampling*.

Maintenant, regardons quelques interactions entre les variables. Nous d'abord ajouter l'ancienneté au dataset.

In [ ]:
# On rajoute l'ancienneté à la table des contrats pour observer des corrélations
tmp1 = pd.merge(
    client[["id_client", "anciennete"]],
    contrat[
        [
            "id_client",
            "id_forfait",
            "facture_mensuelle_moyenne",
            "contrat",
            "resiliation",
        ]
    ],
    on="id_client",
)
tmp1 = pd.merge(tmp1, forfait, on="id_forfait", how="left")
tmp1.head()  # Table provisoire

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(30, 5))
sns.kdeplot(
    data=tmp1, x="anciennete", hue="resiliation", multiple="stack", ax=ax1, cut=0
)
ax1.set(
    xlabel="Ancienneté",
    ylabel="Pourcentage des clients qui résilient",
    title="Résiliation selon l'ancienneté",
)
sns.boxplot(
    data=tmp1.fillna({"go_forfait": "NaN"}),
    x="go_forfait",
    y="facture_mensuelle_moyenne",
    ax=ax2,
    hue="resiliation",
)
ax2.set(
    xlabel="Forfait",
    ylabel="Facture mensuelle moyenne",
    title="Facture mensuelle moyenne par forfait",
)
sns.countplot(data=tmp1, x="contrat", hue="resiliation", ax=ax3)
ax3.set(
    xlabel="Durée du contrat",
    ylabel="Nombre de clients",
    title="Résiliation selon la durée du contrat",
)
sns.countplot(
    data=tmp1.fillna({"go_forfait": "NaN"}),
    x="go_forfait",
    hue="resiliation",
    ax=ax4,
)
ax4.set(
    xlabel="Forfait",
    ylabel="Nombre de clients qui résilient",
    title="Nombre de clients qui résilient selon le forfait",
)
plt.show()

Sur le graphique de gauche, on voit que l'ancienneté des clients a un effet très important sur la résiliation, en effet, les clients anciens ont moins tendance à résilier leur abonnement que les nouveaux clients. 

Dans un deuxième temps, le graphique de droite montre que, quelque soit le forfait, les clients qui ont résilier avaient une facture mensuelle plus faible que les clients qui n'ont pas résilier. Cela peut être dû à des promotions à durée fixe, et donc, une fois cette durée terminée, les clients ont tendance à résilier leur abonnement. Ce qui est en accordance avec le premier graphique.

Ensuite, on peut voir que les données avec le forfait manquant sont très différents des autres en terme de facture mensuelle. En effet, les factures se situent entre le forfait à 20Go et celui à 80Go. On note clairement que la facture mensuelle est très corrélée à la quantité de Go. On peut donc supposer que les forfait manquants correspondent à celui qui n'est pas utilisé et qui contient 50Go. Cette hypothèse sera retenue lors du traitement des valeurs manquantes.

Sur le graphique suivant, on voit que les longs contrats sont moins susceptibles d'être résilier que les contrats courts.

Finalement, le dernier graphique montre que les gros abonnements à 200Go sont plus susceptibles de résilier que les autres abonnements.

### Profil `resiliation_option`

In [ ]:
profil_option.to_notebook_iframe()  # Affiche le profil dans une iframe

La table `resiliation_option` contient les données sur les options ajoutées aux forfaits par les clients, notamment des options de téléphonie, de télévision, de sécurité, de support ou de stream.

Les seules valeurs manquantes sont dans les options de streaming, qui sont corrélées, en effet, si une des options est manquantes, l'autre l'ait également.

On remarque que toutes les options sont très corrélées avec la variable `service_internet`. En effet, si le service internet n'est pas inclu dans le forfait, aucune options ne peut l'être.
Cependant, ce ne sont pas des variables redondantes, car il y a tout de même des options choisies par certains clients.

On peut faire la même remarque pour l'option téléphonie et les lignes multiples.

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(25, 5), sharey=True)
sns.countplot(data=option, x="service_internet", hue="option_securite", ax=axs[0])
axs[0].set(
    xlabel="Service internet",
    ylabel="Nombre de clients",
    title="Service internet selon l'option de sécurité",
)
sns.countplot(data=option, x="service_internet", hue="stream_TV", ax=axs[1])
axs[1].set(
    xlabel="Service internet",
    ylabel="Nombre de clients",
    title="Service internet selon l'option de stream TV",
)
sns.countplot(data=option, x="service_internet", hue="protection_terminal", ax=axs[2])
axs[2].set(
    xlabel="Service internet",
    ylabel="Nombre de clients",
    title="Service internet selon l'option protection terminal",
)
sns.countplot(
    data=option, x="option_service_telephone", hue="multiple_ligne", ax=axs[3]
)
axs[3].set(
    xlabel="Option service téléphonique",
    ylabel="Nombre de clients",
    title="Option service téléphonique selon l'option de multiple ligne",
)
plt.show()

Sur le graphique suivant, on constate encore que les clients avec un forfait manquant sont uniques, en effet, aucun n'ont l'option de service telephonique.
Cela renforce l'idée que les forfaits manquants sont dans une catégorie distincte, plutôt que des valeurs manquantes réparties entre les différents forfaits.

In [ ]:
tmp2 = pd.merge(
    contrat[["id_client", "id_forfait", "facture_mensuelle_moyenne"]],
    option,
    on="id_client",
)
ax = sns.countplot(
    data=tmp2.fillna({"id_forfait": "NaN"}),
    x="id_forfait",
    hue="option_service_telephone",
)
ax.set(
    xlabel="Forfait",
    ylabel="Nombre de clients",
    title="Nombre de clients par forfait en fonction de l'option de service",
)
plt.show()

## Préparation de données

Dans cette partie, nous allons travailler les données et les préparer pour qu'elles puissent être utilisées dans nos modèles.

Nous allons donc traiter le cas des valeurs manquantes et de la transformation des variables catégorielles en variables numériques.

Commençons par regrouper les données dans un dataframe nommé `df` en vérifiant que les id_client sont bien identiques.

In [ ]:
df = pd.merge(
    contrat,
    client,
    on="id_client",
    how="outer",
    validate="1:1",
    indicator="client_contrat",
)
df = pd.merge(
    df, option, on="id_client", how="outer", validate="1:1", indicator="client_option"
)
df = pd.merge(
    df, forfait, on="id_forfait", how="left", validate="m:1", indicator="client_forfait"
)
df.head()

In [ ]:
df["client_contrat"].value_counts()

In [ ]:
df["client_option"].value_counts()

In [ ]:
df["client_forfait"].value_counts()

On constate donc bien que les id_client sont des clés primaires et que les jointures sont bien en *one to one*.

Pour les forfaits, comme mentionné précedemment, on voit qu'il y en a qui sont manquants et qu'un forfait n'est pas du tout présent.

Générons le profil général, qui peut être interressant pour les corrélations.

In [ ]:
profil_total = ProfileReport(df, title="Profil de la dataframe finale")
profil_total.to_file("profils/profil_total.html")

### Traitement des variables

Pour commencer, nous allons évoquer les méthodes possibles pour traiter les différentes varibles.

In [ ]:
df.info()

Nous allons donc regarder variables par variables ce qu'il est necéssaire de faire :

* Tout d'abord, l'`id_client` est une clé primaire, donc elle ne contient pas d'information et il faut donc l'utiliser comme index.
* Les variables `client_contrat`, `client_option` et `client_forfait` ont été créées pour vérifier les clées primaires. Il faut donc les supprimer.
* L'`id_forfait` n'apporte pas plus d'information que la quantité de Go, mais on peut tout de même la garder pour les encoder. De plus, d'après les hypothèses précedentes, on peut supposer que le forfait manquant est celui qui n'est pas utilisé, et donc, mettre la valeur de 50Go.
* La variable `contrat` apporte une information sur la durée du contrat, il semble donc que c'est une variable numérique. On la convertit donc en nombre de mois, quitte à la considérer comme une variable catégorielles par la suite.
* La variable `facture_totale` a quelques valeurs manquantes, on peut les remplir en supposant que la facture totale vaut l'ancienneté fois le facture mensuelle moyenne. Cependant, la facture totale est manquante que quand l'ancienneté vaut 0. On peut donc remplacer par 0.
* Les variables `facture_digitale`, `senior`, `couple`, `parent`, et `option_service_telephone` sont des booléens, on peut donc les convertir en variables numériques avec 0 pour Non et 1 pour Oui.
* Les autres variables ne peuvent pas être converties aussi facilement, effectivement, ce sont des variables catégorielles avec plusieurs valeurs possibles. On peut donc utiliser plusieurs méthodes pour les transformer.
  * On peut notamment utiliser du OneHotEncoding, ce qui est assez simple mais risque de rajouter un nombre important de colonnes redondantes (Pas de service internet notamment).
  * On peut utiliser des Encoders plus complexes, comme ceux de `category_encoders`, tel que `TargetEncoder` qui encode une catégorie en la moyenne de la variable *target*, ce qui *"équivaut à la probabilité"* de résiliation pour la catégorie. Ces estimateurs peuvent ensuite être utilisés sur les données de test en imputant les valeures apprises sur les données d'entrainement.

On va tout d'abord séparer les données en deux groupes : les données d'entrainement et les données de test.

In [ ]:
from sklearn.model_selection import train_test_split

# Dataframe finale
final_df = df.set_index("id_client").drop(
    ["client_contrat", "client_option", "client_forfait"], axis=1
)
# Split data et target
X, y = final_df.drop("resiliation", axis=1), final_df["resiliation"]
# Split data en train et test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Sauvegarde les données générées et csv
os.makedirs("./data/train", exist_ok=True)
os.makedirs("./data/test", exist_ok=True)
X_train.to_csv("./data/train/X_train.csv")
y_train.to_csv("./data/train/y_train.csv")
X_test.to_csv("./data/test/X_test.csv")
y_test.to_csv("data/test/y_test.csv")

Maintenant, nous allons utiliser `sklearn-pandas` pour associer un ou plusieurs estimateurs pour chaque variable catégorielles, afin de les transformer en features utilisable par nos modèles.

In [ ]:
# def preprocessing(df):
#     """Cette fonction permet de pré-traiter les données.

#     Args:
#         data (DataFrame): La dataframe à pré-traiter sans la colonne target
#             et avec les id_client en indice. Cette fonction doit fonctionner
#             à la fois sur les données d'entrainement et de test.

#     Returns:
#         DataFrame: La dataframe pré-traitée.
#     """
#     # On copie la dataframe pour ne pas la modifier
#     data = df.copy()
#     data = data.drop("id_forfait", axis=1)  # On enlève la colonne id_forfait
#     # On remplace les valeurs booléennes et numériques par des entiers
#     data = data.replace(
#         {
#             "contrat": {"Mensuel": 1, "un an": 12, "deux ans": 24},
#             # "facture_digitale": {"Oui": 1, "Non": 0},
#             # "senior": {"Oui": 1, "Non": 0},
#             # "couple": {"Oui": 1, "Non": 0},
#             # "parent": {"Oui": 1, "Non": 0},
#             # "option_service_telephone": {"Oui": 1, "Non": 0},
#         }
#     )
#     # On remplace les valeurs manquantes dans les Go par 50.
#     data = data.fillna({"go_forfait": 50})
#     # On remplace les valeurs manquantes dans les factures totales par 0.
#     data = data.fillna({"facture_totale": 0})
#     return data
#
#
# def final_processing(df, mapper):
#     """Cette fonction est un wrapper pour les fonctions de preprocessing.

#     Elle applique le mapper et supprime les colonnes redondantes issues du OneHotEncoding.

#     Args:
#         data (DataFrame): La dataframe à finalement transformer.
#         mapper (DataFrameMapper): Le mapper fitter auparavant qui a été utilisé pour transformer les données.

#     Returns:
#         DataFrame: La dataframe finalement transformée.
#     """
#     # On copie la dataframe pour ne pas modifier la dataframe d'origine
#     data = df.copy()
#     # On applique le premier processing
#     # data = preprocessing(data)
#     # On transforme les données avec le mapper
#     data = mapper.transform(data)
#     # On supprime les colonnes redondantes
#     data = data.drop(data.filter(regex="Pas de service internet").columns, axis=1)
#     data = data.drop(data.filter(regex="Pas de ligne telephonique").columns, axis=1)
#     return data

In [ ]:
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.preprocessing import (
    LabelEncoder,
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    PowerTransformer,
    QuantileTransformer,
    StandardScaler,
)
from sklearn_pandas import DataFrameMapper

# Mapper qui permet de transformer les colonnes en colonnes numériques
# La stratégie de transformation est la plus simple pour le moment, pour les données manquantes,
# on impute avec la valeur la plus fréquente. Pour les données categoriales, on utilise du OneHotEncoding.
# Pour les données numériques, on utilise des transformations standards (StandardScaler, PowerTransformer, MinMaxScaler).
mapper = DataFrameMapper(
    [
        (
            ["id_forfait"],
            [
                SimpleImputer(strategy="constant", fill_value="Dktt"),
                OneHotEncoder(handle_unknown="ignore"),
            ],
        ),
        (["go_forfait"], SimpleImputer(strategy="constant", fill_value=50)),
        (["contrat"], OneHotEncoder(handle_unknown="ignore")),
        (["facture_digitale"], OneHotEncoder(handle_unknown="ignore", drop="first")),
        (["methode_paiement"], OneHotEncoder(handle_unknown="ignore")),
        (["facture_mensuelle_moyenne"], MinMaxScaler()),
        (
            ["facture_totale"],
            [SimpleImputer(strategy="constant", fill_value=0), PowerTransformer()],
        ),
        (
            ["genre"],
            [
                SimpleImputer(strategy="most_frequent"),
                OneHotEncoder(handle_unknown="ignore"),
            ],
        ),
        (["senior"], OneHotEncoder(handle_unknown="ignore")),
        (["couple"], OneHotEncoder(handle_unknown="ignore")),
        (
            ["parent"],
            [
                SimpleImputer(strategy="most_frequent"),
                OneHotEncoder(handle_unknown="ignore", drop="first"),
            ],
        ),
        (["anciennete"], MinMaxScaler()),
        (["option_service_telephone"], OneHotEncoder(handle_unknown="ignore")),
        (["multiple_ligne"], OneHotEncoder(handle_unknown="ignore", drop="first")),
        (["service_internet"], OneHotEncoder(handle_unknown="ignore")),
        (["option_securite"], OneHotEncoder(handle_unknown="ignore", drop="first")),
        (["option_backup"], OneHotEncoder(handle_unknown="ignore", drop="first")),
        (["protection_terminal"], OneHotEncoder(handle_unknown="ignore", drop="first")),
        (["support_technique"], OneHotEncoder(handle_unknown="ignore", drop="first")),
        (
            ["stream_TV"],
            [
                SimpleImputer(strategy="most_frequent"),
                OneHotEncoder(handle_unknown="ignore", drop="first"),
            ],
        ),
        (
            ["stream_Films"],
            [
                SimpleImputer(strategy="most_frequent"),
                OneHotEncoder(handle_unknown="ignore", drop="first"),
            ],
        ),
    ],
    df_out=True,
)

# On ajoute un column transformer pour supprimer les colonnes redondantes
ct = ColumnTransformer(
    [
        (
            "drop_internet",
            "drop",
            make_column_selector(pattern="Pas de service internet"),
        ),
        (
            "drop_telephonique",
            "drop",
            make_column_selector(pattern="Pas de ligne telephonique"),
        ),
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
)

mapper1 = make_pipeline(mapper, ct)

Par exemple, si on applique ce mapper au dataset `X_train`, on obtient :

In [ ]:
pd.DataFrame(
    mapper1.fit_transform(X_train),
    columns=ct.get_feature_names_out(),
    index=X_train.index,
).head()

Il est également possible de faire un autre mapper afin d'avoir un autre encoding.

In [ ]:
# Ce mapper utilise du target encoding, il faut donc fit avec la target. De plus, TargetEncoder renvoie la moyenne s'il y a des données manquantes.
mapper2 = DataFrameMapper(
    [
        (
            ["id_forfait"],
            [SimpleImputer(strategy="constant", fill_value="Dktt"), TargetEncoder()],
        ),
        (["go_forfait"], SimpleImputer(strategy="constant", fill_value=50)),
        (["contrat"], TargetEncoder()),
        (["facture_digitale"], TargetEncoder()),
        (["methode_paiement"], TargetEncoder()),
        (["facture_mensuelle_moyenne"], None),
        (["facture_totale"], SimpleImputer(strategy="constant", fill_value=0)),
        (["genre"], TargetEncoder()),
        (["senior"], TargetEncoder()),
        (["couple"], TargetEncoder()),
        (["parent"], TargetEncoder()),
        (["anciennete"], None),
        (["option_service_telephone"], TargetEncoder()),
        (["multiple_ligne"], TargetEncoder()),
        (["service_internet"], TargetEncoder()),
        (["option_securite"], TargetEncoder()),
        (["option_backup"], TargetEncoder()),
        (["protection_terminal"], TargetEncoder()),
        (["support_technique"], TargetEncoder()),
        (["stream_TV"], TargetEncoder()),
        (["stream_Films"], TargetEncoder()),
    ],
    df_out=True,
)

Ce mapper renvoie une dataframe très différente, en effet, les colonnes ne sont pas dupliquées pour faire du OneHotEncoding, de plus la stratégie pour les valeurs manquantes est différente. Elle sont remplacées par la valeurs moyenne de la variable target.

In [ ]:
mapper2.fit_transform(X_train, LabelEncoder().fit_transform(y_train))

## Modélisation

Nous allons maintenant passer à la modélisation. Dans un premier temps, nous mettrons en place un modèle de base (régression logistique) pour avoir un premier score.

La métrique utilisée sera le **recall**. En effet, le recall semble important car il décrit le nombre de clients qui résilient, et qui n'ont pas été prédit comme tel par le modèle. Cela est donc utile si l'objectif est de conserver les clients le plus possible.

Par exemple, si on veut prévoir des clients à integrer dans une campagne de démarchage, ce modèle fournira la majorité des clients qui sont très suceptibles de résilier, en plus d'une quantité non négligeable de clients qui n'avait pas l'intention de résilier. Mais cela permet quand même de réduire drastiquement le scope de l'opération de démarchage.

Mais, il reste important de regarder la précision pour ne pas tout prédire comme résilient. Pour cela, on peut regarder le **f1 score**.

In [ ]:
# On fit sur les données de train. Il est également possible de les rajouter dans des pipelines.
le = LabelEncoder().fit(y_train)
mapper1.fit(X_train, le.transform(y_train))
mapper2.fit(X_train, le.transform(y_train))
# X_train_processed = mapper1.transform(X_train)
# X_test_processed = mapper1.transform(X_test)
# y_train_processed = le.transform(y_train)
# y_test_processed = le.transform(y_test)

In [ ]:
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
)


# Fonctions pour afficher la matrice de confusion
def plot_confusion_matrix(y_true, y_pred, label_encoder):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ConfusionMatrixDisplay.from_predictions(
        y_true,
        y_pred,
        display_labels=label_encoder.classes_,
        ax=ax1,
        cmap="Blues",
        normalize="true",
        values_format=".0%",
    )
    ax1.grid(False)
    ConfusionMatrixDisplay.from_predictions(
        y_true, y_pred, display_labels=label_encoder.classes_, ax=ax2, cmap="Blues"
    )
    ax2.grid(False)
    plt.show()


# Fonction pour afficher la matrice de confusion ainsi que certains scores
def plot_results(
    y_true,
    y_pred,
    label_encoder,
    metrics=[f1_score, roc_auc_score, precision_score, recall_score],
):
    for metric in metrics:
        print(metric.__name__, metric(y_true, y_pred))
    plot_confusion_matrix(y_true, y_pred, label_encoder)

In [ ]:
# Fonction qui entraine un modèle après appliquer un mapper et renvoie les scores et la matrice de confusion
def test_model(model, mapper, X_train, y_train, X_test, y_test, label_encoder):
    pipe = make_pipeline(mapper, model)
    pipe.fit(X_train, label_encoder.transform(y_train))
    y_true, y_pred = label_encoder.transform(y_test), pipe.predict(X_test)
    plot_results(y_true, y_pred, label_encoder)
    return pipe


# Ajoute le sampling
def test_model_with_sampling(
    model, mapper, sampler, X_train, y_train, X_test, y_test, label_encoder
):
    y_train_processed = label_encoder.transform(y_train)
    X_train_processed = mapper.fit_transform(X_train, y_train_processed)
    X_test_processed = mapper.transform(X_test)
    X_res, y_res = sampler.fit_resample(X_train_processed, y_train_processed)
    clf = model.fit(X_res, y_res)
    y_true, y_pred = label_encoder.transform(y_test), clf.predict(X_test_processed)
    plot_results(y_true, y_pred, label_encoder)
    return clf

### Modèle de base

Premièrement, nous allons utiliser un modèle de base, afin d'avoir une idée de la performance d'un modèle simple, en l'occurence, un modèle de régression logistique.

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42)
print("Pour le mapper 1")
log_reg_1 = test_model(clf, mapper1, X_train, y_train, X_test, y_test, le)
print("Pour le mapper 2")
log_reg_2 = test_model(clf, mapper2, X_train, y_train, X_test, y_test, le)

On voit donc que les deux preprocessing des données donnent des résultats similaires pour une regression logistique (le mapper 1 fonctionne un tout petit peu mieux que le mapper 2).

Le score f1 est à 0.63, ce qui est assez moyen, et surtout, le recall est très faible. Sur les 373 clients qui ont résilié dans le test set, il y a seulement 217 clients qui ont été prédit comme tel. Ce qui fait que l'on perdrait beacoup de clients.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=42)
print("Pour le mapper 1")
rf_1 = test_model(clf, mapper1, X_train, y_train, X_test, y_test, le)
print("Pour le mapper 2")
rf_2 = test_model(clf, mapper2, X_train, y_train, X_test, y_test, le)

Ce problème est encore pire si on regarde un RandomForest classique.

Il faut donc essayer de gérer le problème de déséquilibre des classes. Pour cela, on peut utiliser des technique de sampling qui enlève des données de la classes dominantes de manière aléatoire, ou rajoute des observations.
Pour ce faire, on va utiliser le package `imblearn` qui contient des stratégies de sampling.

In [ ]:
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import ADASYN, SMOTE, BorderlineSMOTE
from imblearn.under_sampling import TomekLinks

smote = SMOTE(random_state=42)
smoteen = SMOTEENN(random_state=42)
tomek = TomekLinks()

clf = LogisticRegression(random_state=42)
print("Pour le mapper 1 avec SMOTE")
lr_smote = test_model_with_sampling(
    clf, mapper1, smote, X_train, y_train, X_test, y_test, le
)
print("Pour le mapper 1 avec SMOTEEN")
lr_smoteen = test_model_with_sampling(
    clf, mapper1, smoteen, X_train, y_train, X_test, y_test, le
)
print("Pour le mapper 1 avec TomekLinks")
lr_smoteen = test_model_with_sampling(
    clf, mapper1, tomek, X_train, y_train, X_test, y_test, le
)

On voit clairement, que cette technique permet d'améliorer de manière significative le recall, qui passe d'environ 50% à presque 90% avec le sampler SMOTEEN. Cepandant, la précision diminue significativement et le score f1 reste autour de 0.6.

Dans la suite, nous allons utiliser le sampler SMOTEEN.

### Essai de XGBoost, LightGBM, réseaux de neurones ...

Ici, nous allons de tester différents modèles de classification, pour voir s'il est possible d'améliorer le modèle de base.

In [ ]:
from xgboost import XGBClassifier

clf = XGBClassifier(random_state=42)
print("Essai avec XGBClassifier et SMOOTEEN pour le mapper 1")
xgb_1 = test_model_with_sampling(
    clf, mapper1, smoteen, X_train, y_train, X_test, y_test, le
)
print("Essai avec XGBClassifier et SMOOTEEN pour le mapper 2")
xgb_2 = test_model_with_sampling(
    clf, mapper2, smoteen, X_train, y_train, X_test, y_test, le
)

In [ ]:
import lightgbm as lgb

clf = lgb.LGBMClassifier(random_state=42)
print("Essai avec LGBMClassifier et SMOOTEEN pour le mapper 1")
lgb_1 = test_model_with_sampling(
    clf, mapper1, smoteen, X_train, y_train, X_test, y_test, le
)
print("Essai avec LGBMClassifier et SMOOTEEN pour le mapper 2")
lgb_2 = test_model_with_sampling(
    clf, mapper2, smoteen, X_train, y_train, X_test, y_test, le
)

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=42)
print("Essai avec MLPClassifier et SMOOTEEN pour le mapper 1")
mlp_1 = test_model_with_sampling(
    clf, mapper1, smoteen, X_train, y_train, X_test, y_test, le
)
print("Essai avec MLPClassifier et SMOOTEEN pour le mapper 2")
mlp_2 = test_model_with_sampling(
    clf, mapper2, smoteen, X_train, y_train, X_test, y_test, le
)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(random_state=42)
print("Essai avec AdaBoostClassifier et SMOOTEEN pour le mapper 1")
ada_1 = test_model_with_sampling(
    clf, mapper1, smoteen, X_train, y_train, X_test, y_test, le
)
print("Essai avec AdaBoostClassifier et SMOOTEEN pour le mapper 2")
ada_2 = test_model_with_sampling(
    clf, mapper2, smoteen, X_train, y_train, X_test, y_test, le
)

Ainsi, l'estimateur AdaBoost, la regression logistique et le réseau de neurones semblent les plus performant en terme de recall, cependant, AdaBoost a un meilleur score f1 que les deux autres. Nous allons donc utiliser celui-ci, avec le mapper 1 et le sampler SMOTEEN.

### Optimisation des hyperparamètres

Maintenant, nous allons voir s'il est possible d'améliorer le modèle en optimisant les hyperparamètres.

In [ ]:
from sklearn.model_selection import GridSearchCV

gscv = GridSearchCV(
    estimator=AdaBoostClassifier(random_state=42),
    param_grid={
        "n_estimators": [50, 100, 200, 400],
        "learning_rate": [0.1, 0.5, 1.0],
    },
    scoring="recall",
    cv=5,
    n_jobs=-1,
)

In [ ]:
y_train_processed = le.transform(y_train)
X_train_processed = mapper1.fit_transform(X_train, y_train_processed)
X_test_processed = mapper1.transform(X_test)
X_res, y_res = smoteen.fit_resample(X_train_processed, y_train_processed)
gscv.fit(X_res, y_res)

In [ ]:
gscv.best_estimator_.get_params()

On voit que les meilleurs paramètres sont différents des valeurs par défaut, qui sont 50 estimateurs et 1.0 pour le paramètre `learning_rate`.

In [ ]:
# Matrice de confusion pour le modèle issu de la GridSearch
print("Score du modèle AdaboostClassifier sélectionné avec GridSearch")
y_true, y_pred = le.transform(y_test), gscv.predict(X_test_processed)
plot_results(y_true, y_pred, le)

In [ ]:
# Modèle par défaut qui a été sélectionné plus haut.
print("Score du modèle AdaboostClassifier précédemment sélectionné")
test_model_with_sampling(
    AdaBoostClassifier(
        **{
            "algorithm": "SAMME.R",
            "base_estimator": None,
            "learning_rate": 1.0,
            "n_estimators": 50,
        }
    ),
    mapper1,
    smoteen,
    X_train,
    y_train,
    X_test,
    y_test,
    le,
);

Cependant, quand on compare les deux matrices de confusion, on voit que le premier modèle est légerement meilleur que le second. Par contre, le modèle issu de la GridSearch a un meilleur score f1.
Il reste à voir si la performance ajoutée avec ce modèle est suffisante pour compenser la complexité fortement accrue. En effet, le modèle de classification le plus simple fonctionnait déjà très bien, il faut voir si on peut l'optimiser un peu et s'il devient meilleur.

In [ ]:
print("Score de la regression logistique")
lr_smoteen = test_model_with_sampling(
    LogisticRegression(random_state=42),
    mapper1,
    smoteen,
    X_train,
    y_train,
    X_test,
    y_test,
    le,
)

In [ ]:
gscv2 = GridSearchCV(
    estimator=LogisticRegression(random_state=42, n_jobs=-1),
    param_grid={
        "C": [0.1, 0.5, 1.0, 5.0],
        "max_iter": [100, 200, 500],
        "penalty": ["l1", "l2", "none"],
        "fit_intercept": [True, False],
    },
    scoring=["recall", "f1"],
    refit="f1",
    cv=5,
    n_jobs=-1,
)
gscv2.fit(X_res, y_res)
gscv2.best_estimator_.get_params()

In [ ]:
# Matrice de confusion pour le modèle de regression logistique issu de la GridSearch
print("Score de la regression logistique sélectionnée avec GridSearch")
plot_results(le.transform(y_test), gscv2.predict(X_test_processed), le)

Au final, on vient bien que la régression linéaire est très performante bien que c'est un modèle très simple. Je vais donc choisir ce modèle car il sera plus simple à expliquer et à interpréter.

### Explicabilité du modèle sélectionné

Dans cette partie, nous allons nous pencher plus en détails sur les variables qui ont un impact sur les décisions du modèle. Pour ce faire, nous allons utiliser le package `explainerdashboard` et `shap`.

In [ ]:
from explainerdashboard import ClassifierExplainer, ExplainerDashboard

cats = [
    {
        "id_forfait": [
            "id_forfait_x0_Dkgo",
            "id_forfait_x0_Dkji",
            "id_forfait_x0_Dkop",
            "id_forfait_x0_Dktt",
            "id_forfait_x0_Dkwg",
        ]
    },
    {"contrat": ["contrat_x0_Mensuel", "contrat_x0_deux ans", "contrat_x0_un an"]},
    {
        "methode_paiement": [
            "methode_paiement_x0_Carte de credit Automatique",
            "methode_paiement_x0_Cheque electronique",
            "methode_paiement_x0_Cheque par mail",
            "methode_paiement_x0_RIB Automatique",
        ]
    },
    {"genre": ["genre_x0_Madame", "genre_x0_Mademoiselle", "genre_x0_Monsieur"]},
    {"senior": ["senior_x0_Non", "senior_x0_Oui"]},
    {"couple": ["couple_x0_Non", "couple_x0_Oui"]},
    {
        "option_service_telephonique": [
            "option_service_telephone_x0_Non",
            "option_service_telephone_x0_Oui",
        ]
    },
    {
        "service_internet": [
            "service_internet_x0_DSL",
            "service_internet_x0_Fibre",
            "service_internet_x0_Non",
        ]
    },
]
tmp = pd.DataFrame(
    X_test_processed, columns=ct.get_feature_names_out(), index=y_test.index
)
explainer = ClassifierExplainer(
    gscv2.best_estimator_,
    tmp,
    le.transform(y_test),
    cats=cats,
    labels=le.classes_.tolist(),
    idxs=y_test.index,
    index_name="Id Client",
    target="Résiliation",
    n_jobs=-1,
)

db = ExplainerDashboard(explainer, title="Resiliation Model Explainer")

In [ ]:
db.run(mode="inline")

Ce dashboard interactif permet les shap values et donc l'importance des différentes variables sur le modèle.

Dans ce cas, on voit que les factures et l'ancienneté sont les variables les plus importantes pour décider si un client est résilier ou non. Il faut quand même noter que ces variables sont très corrélées, et donc interchangeables, car la facture totale est presque égale au produit de la facture mensuelle moyenne et de l'ancienneté.

Sinon, on voit que la durée du contrat est également très importante.

D'un autre côté, les variables genre, senior et couple ont un impact très faibles sur le modèle, ce qui peut être étonnant, surtout pour la situation conjuguale.

Ce dashboard permet également de voir les shap values pour les clients individuellement et ainsi de comprendre comment l'algorithme prend la décision. On peut aussi, changer la valeur d'un paramètre d'un client pour voir si la décision change.

Le graphique **Precision Plot** dans l'onglet **Classification Stats** montre que les décisions sont très souvent bien claires et que la majorité des clients qui résilient sont bien classés. Pour encore améliorer le score, on aurait pu baisser le threshold en dessous de 0.5.

On y voit aussi les courbes de trade-off entre la précision et le recall. Et on voit très clairement que le recall est privilégié par rapport à la précision. Ce que se traduit par une assez mauvaise classification des clients ne souhaitant pas résilier. Cependant, ce n'était pas mon objectif. 

#### Shap Beeswarm

Pour comprendre l'impact positif ou négatif des variables, le package `shap` fournit le graphique Beeswarm qui illustre bien cela.

In [ ]:
import shap

shap.initjs()

explainer = shap.Explainer(gscv2.best_estimator_, tmp)
shap_values = explainer(tmp)

In [ ]:
shap.plots.beeswarm(shap_values, order=shap_values.abs.max(0), max_display=20)

Ce graphique montre qu'une facture totale élévée a un impact négatif sur la décision du modèle, à savoir que le client va résilier. Ceci s'explique par le fait que les clients avec une facture totale élevée sont anciens et ont moins de chance de résilier.

Au contraire, une facture mensuelle faible se traduit par plus de résiliation, il y a probablement plusieurs facteurs pouvant expliquer cela, on peut supposer que le fait que les clients avec un "petit" forfait ont plus tendance à en changer.

Le type de contrat à aussi un impact, par exemple, les longs contrats (de 2 ans) sont préférables à ceux de moins de 1 mois.

## Conclusion

Ainsi, on a construit un modèle de classification dont l'objectif principal est de déterminer si un client va résilier, dans l'optique d'essayer de le conserver en lui proposant des offres ou en faisant un démarchage plus ciblé. Pour cette tâche l'algorithme utilisé est performant, bien qu'il soit très simple. Il est donc facile d'expliquer les décisions avec les shap values notamment.

#### Points à améliorer ou à approfondir

Je vais finir par proposer des pistes d'améliorations ou de développements plus complexes.

* Premièrement, il faudrait voir si on peut améliorer la précision du modèle tout en gardant le recall faible.
* Pour ce faire, il semble pertinent de tester différentes méthodes de preprocessing des données, des méthodes d'*Imputing* plus adaptée notamment. Et de voir si on peut ajouter d'autres sources.
* Il faut aussi tester d'autres modèles qui peuvent être plus performants.
* Pour cela, on peut aussi améliorer l'optimisation des hyperparamètres, avec une librairie comme `optuna` à la place de GridSearch CV par exemple.

Repo GitHub du projet: https://github.com/gwatkinson/test-technique-bouygues